<a href="https://colab.research.google.com/github/mapsguy/programming-gemini/blob/main/veo2_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import the genai library
from google import genai

In [2]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [5]:
import time
from google.genai import types

Text to Video Generation

In [ ]:
#I can't run this

prompt_text = "A majestic eagle soaring through a canyon at sunset, cinematic lighting."

gen_config = types.GenerateVideosConfig(
    duration_seconds=10.0,
    #style_preset="STYLE_PRESET_CINEMATIC"
)

try:
    video_operation = client.models.generate_videos(
        model="veo-2.0-generate-001", # Use the appropriate Veo model identifier
        prompt=prompt_text,
        config=gen_config
    )
    print(f"Video generation started. Operation ID: {video_operation.operation.name}")

    print("Polling for completion...")
    while not video_operation.done():
        print("Waiting for 10 seconds...")
        time.sleep(10)
        video_operation = client.operations.get(video_operation)
        # You can also refresh the operation state if needed,
        # though the object often updates itself or has methods to wait.
        # For explicit polling, refer to the specific Operation object's methods.

    #download the video
    for n, generated_video in enumerate(video_operation.response.generated_videos):
      client.files.download(file=generated_video.video)
      generated_video.video.save(f"video{n}.mp4")  # save the video



except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
#display the video
from IPython.display import Video

Video("video-path.mp4", embed=True)

Image to Video Generation: Combining Imagen 3 and Veo Use

In [ ]:
#Not able to test this

#provide an initial image and a prompt to create a video sequence

#prompt for initial image
prompt_for_image = "A landscape, with clouds slowly drifting and subtle light changes."

image_config=types.GenerateImagesConfig(
    number_of_images=1,
    aspect_ratio="1:1"
)

imagen = client.models.generate_images(
    model="imagen-3.0-generate-002",
    prompt=prompt_for_image,
    config=image_config
)

image_gcs_uri = imagen.images[0].gcs_uri

prompt_for_video="Animate this landscape, with clouds slowly drifting and subtle light changes."

vid_config=types.GenerateVideosConfig(
    duration_seconds=8.0,
    #style_preset="STYLE_PRESET_DIGITAL_ART",
    number_of_videos=2,
    person_generation="dont_allow"
)

In [ ]:
#not able to test

try:
    video_operation_img = client.models.generate_videos(
        model="imagen-3.0-generate-002",
        prompt=prompt_for_video,
        image=imagen.generated_images[0].image,
        config=vid_config
        #image_gcs_uri=image_gcs_uri
    )

    print(f"Image-to-video generation started. Operation ID: {video_operation_img.operation.name}")

    while not video_operation_img.done():
        time.sleep(10)
        video_operation_img = client.operations.get(video_operation_img)

    for n, video in enumerate(video_operation_img.response.generated_videos):
      fname = f'with_image_input{n}.mp4'
      print(fname)
      client.files.download(file=video.video)
      video.video.save(fname)
      print(f"Generated video from image URI: {image_gcs_uri}")

except Exception as e:
    print(f"An error occurred during image-to-video generation: {e}")